Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)

BSD license

python 3에서 실행 가능하도록 수정, 한글 해설 추가

In [25]:
import numpy as np

In [26]:
#데이터를 불러오고, 글자-벡터 간 상호 변환 매핑 준비 
data = open('input.txt', 'r').read() # 텍스트 파일 로드 [ 디렉토리 내에 input.txt 파일이 있으면 불러올 수 있음.]
chars = list(set(data)) # 텍스트 파일에서 고유한 문자 추출 

### set => 집합 자료형

순서가 없고 중복을 허용하지 않는다는 특징을 가진다. 중복을 허용하지 않는다는 특징은 프로그래밍에서 매우 유용하게 사용된다.

In [27]:
# 집합 선언
set1 = set([1,2,3])
set2 = set("Conor Mcgregor")
 
print(set1)
print(set2)

{1, 2, 3}
{'c', ' ', 'n', 'C', 'o', 'e', 'g', 'r', 'M'}


In [28]:
list(set([1,2,3])),list(set("Conor Mcgregor"))

([1, 2, 3], ['c', ' ', 'n', 'C', 'o', 'e', 'g', 'r', 'M'])

In [29]:
data_size, vocab_size = len(data), len(chars) # 길이
print('데이터는 {}개의 글자로 되어 있고, {}개의 고유한 문자가 있습니다.'.format(data_size, vocab_size))
print(repr(''.join(sorted(str(x) for x in chars)))) # 추출된 고유한 글자들을 알파벳 순서대로 출력

데이터는 355개의 글자로 되어 있고, 110개의 고유한 문자가 있습니다.
'\n ,.02‘’가간개것결경계과관교구국그기나는다대던데도됐드듭략러려력련로른를마매면목문미반발변보봉부사살상색서선성수시식안양었에열영외요우위유은을의이인입있장적전점정제주준중지진짓체축출측토통편포표필하한할합해했협호'


### str함수와 repr함수

In [30]:
a = 123
str(a),repr(a)

('123', '123')

In [31]:
a = "Life is too short"
str(a),repr(a)

('Life is too short', "'Life is too short'")

### repr은 단일인용부호(')가 좌우로 감싸여진 형태의 문자열을 리턴해 주었다.
str의 사용목적 : "사용자가 보기 쉽게 하기 위해"

repr의 사용목적 : "문자열로 객체를 다시 생성할 수 있기 위해"

자세한 사항은 https://wikidocs.net/11741 참고

In [32]:
# 고유한 글자들(a,b,c,d...)을 숫자(1,2,3,4...)에 매핑하는 사전과, 반대 기능을 수행하는 사전을 만듦
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [34]:
char_to_ix

{'\n': 73,
 ' ': 17,
 ',': 36,
 '.': 19,
 '0': 68,
 '2': 46,
 '‘': 25,
 '’': 37,
 '가': 56,
 '간': 70,
 '개': 104,
 '것': 30,
 '결': 90,
 '경': 57,
 '계': 107,
 '과': 55,
 '관': 103,
 '교': 79,
 '구': 4,
 '국': 109,
 '그': 9,
 '기': 66,
 '나': 95,
 '는': 86,
 '다': 10,
 '대': 105,
 '던': 59,
 '데': 97,
 '도': 96,
 '됐': 2,
 '드': 67,
 '듭': 99,
 '략': 50,
 '러': 53,
 '려': 85,
 '력': 11,
 '련': 100,
 '로': 92,
 '른': 3,
 '를': 32,
 '마': 16,
 '매': 65,
 '면': 64,
 '목': 93,
 '문': 94,
 '미': 18,
 '반': 75,
 '발': 45,
 '변': 38,
 '보': 15,
 '봉': 24,
 '부': 28,
 '사': 40,
 '살': 108,
 '상': 27,
 '색': 14,
 '서': 78,
 '선': 60,
 '성': 39,
 '수': 12,
 '시': 89,
 '식': 31,
 '안': 34,
 '양': 41,
 '었': 29,
 '에': 13,
 '열': 91,
 '영': 33,
 '외': 22,
 '요': 83,
 '우': 42,
 '위': 48,
 '유': 69,
 '은': 101,
 '을': 72,
 '의': 6,
 '이': 74,
 '인': 77,
 '입': 71,
 '있': 61,
 '장': 23,
 '적': 58,
 '전': 7,
 '점': 80,
 '정': 1,
 '제': 26,
 '주': 8,
 '준': 63,
 '중': 81,
 '지': 106,
 '진': 0,
 '짓': 5,
 '체': 52,
 '축': 21,
 '출': 44,
 '측': 82,
 '토': 54,
 '통': 62,
 '편': 76,
 '포': 35,


### enumerate함수

enumerate는 "열거하다"라는 뜻. 이 함수는 순서가 있는 자료형(리스트, 튜플, 문자열)을 입력으로 받아 인덱스 값을 포함하는 enumerate 객체를 리턴한다.

In [33]:
for i, name in enumerate(['body', 'foo', 'bar']):
    print(i, name)

0 body
1 foo
2 bar


순서값과 함께 body, foo, bar가 순서대로 출력되었다.

즉, 위 예제와 같이 enumerate를 for문과 함께 사용하면 자료형의 현재 순서(index)와 그 값을 쉽게 알 수 있다.

for문처럼 반복되는 구간에서 객체가 현재 어느 위치에 있는지 알려주는 인덱스 값이 필요할때 enumerate 함수를 사용하면 매우 유용하다.

In [35]:
# 하이퍼파라미터 설정
hidden_size = 100 # hidden state의 뉴런 갯수
seq_length = 25 # 학습시킬 때 한번에 불러올 글자 수이자 RNN을 펼쳤을 때의 단계 
learning_rate = 1e-1 # 학습속도, 가중치를 조정할 때 이동할 간격


In [36]:
# 모델 파라미터 초기화(가중치는 작은 수의 랜덤한 값, bias는 0으로 초기화)
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden (100,25)
#vocab_size : 고유문자 개수 [Wxh : x와 h의 가중치]
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden (100,100)
#[Whh : h와 h의 가중치]
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output (25,100)
#[Why : h와 y의 가중치]

### NumPy의 random 서브패키지에는 난수를 생성하는 명령이 3가지가 있다.

rand: 0부터 1사이의 균일 분포

randn: 가우시안 표준 정규 분포

randint: 균일 분포의 정수 난수

#### rand 명령은 0부터 1사이에서 균일한 확률 분포로 실수 난수를 생성한다.

숫자 인수는 생성할 난수의 크기이다. 여러개의 인수를 넣으면 해당 크기를 가진 행렬을 생성한다.

In [16]:
np.random.rand(10)

array([ 0.93513181,  0.63723792,  0.45709797,  0.86082746,  0.53882693,
        0.16729639,  0.5199929 ,  0.92752859,  0.08082023,  0.63179954])

In [17]:
np.random.rand(3, 5)

array([[ 0.14773079,  0.67698661,  0.83980192,  0.62959153,  0.73523859],
       [ 0.30682033,  0.9198811 ,  0.84723781,  0.59230736,  0.45690183],
       [ 0.35653358,  0.35440135,  0.67222602,  0.99690964,  0.54468561]])

In [18]:
np.random.randn(10)

array([-1.73239222, -1.11319852, -0.15825056, -0.60810003, -1.29147185,
        0.6911161 , -0.21898878,  0.41383942,  0.0825576 , -0.47295713])

In [38]:
bh = np.zeros((hidden_size, 1)) # hidden bias (100,1)
by = np.zeros((vocab_size, 1)) # output bias (25,1)


#### np.zeros(), np.ones(), np.empty() 함수는 괄호 안에 쓴 숫자 개수만큼의 '0', '1', '비어있는 배열' 공간을 만들어줍니다.


In [37]:
np.zeros(5)

array([ 0.,  0.,  0.,  0.,  0.])

### 일반 rnn 코드

In [ ]:
#https://ratsgo.github.io/natural%20language%20processing/2017/03/09/rnnlstm/
#참고

def lossFun(inputs, targets, hprev):
    """
    inputs, targets는 모두 숫자 인덱스의 리스트이다.
    hprev는 H(hidden_size)x1의 array, 이전 학습에서 반환한 마지막 hidden state임.
    forward pass(손실값 계산), backward pass(그래디언트 계산)를 모두 수행한 후 
    손실값, 각각의 가중치에 대한 그래디언트, 그리고 다음 반복 때 사용할 마지막 hidden state를 반환함
    """
    
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass(손실값 계산)
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # 1-of-k(one-hot) 형태로 변환. 모든 값이 0인 array 준비
        xs[t][inputs[t]] = 1 # 해당하는 글자에만 값을 1로 설정 - [0, ..., 0, 1, 0, ..., 0]
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state 업데이트
        #np.dot : Numpy에선 벡터의 내적, 벡터와 행렬의 곱, 행렬곱을 위해 ‘‘대신 ‘dot’함수를 사용합니다.
        #         dot’은 Numpy 모듈 함수로서도 배열 객체의 인스턴스 메소드로서도 이용 가능한 함수입니다:
        #np.tanh : # TanH 함수: 쌍곡함수 (hyperbolic function)
        
        ys[t] = np.dot(Why, hs[t]) + by # 다음 글자가 어떤 글자가 나올지에 가능성을 표시한 array(정규화되지 않음)
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # softmax로 각 글자의 등장 가능성을 확률로 표시
        #np.exp() 함수는 밑(base)이 자연상수 e 인 지수함수로 변환해줍니다.
        
        #softmax는 데이터를 2개 이상의 그룹으로 나누기 위해 binary classification을 확장한 모델이다.
        #softmax : http://pythonkim.tistory.com/19  참고
        
        loss += -np.log(ps[t][targets[t],0]) # cross-entropy를 이용하여 정답과 비교하여 손실값 판정
        #cross-entropy는 통계학 용어로, 두 확률 분포 p와 q 사이에 존재하는 정보량을 계산하는 방법을 말한다. 

        
    # backward pass(그래디언트 계산)[역전파]
    # 변수 초기화
    
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    #만약 크기를 튜플(tuple)로 명시하지 않고 특정한 배열 혹은
    #리스트와 같은 크기의 배열을 생성하고 싶다면 ones_like, zeros_like 명령을 사용한다.
    
    for t in reversed(range(len(inputs))): #forward pass의 과정을 반대로 진행(t=24부터 시작)
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # y의 그래디언트 계산, softmax 함수의 그래디언트 계산
        
        
        #할당연산자
        # -=, +=, *=, /=
        # a -= 3 => a = a - 3 
        # a += 3 => a = a + 3
        # a *= 3 => a = a * 3 
        
        
        dWhy += np.dot(dy, hs[t].T) 
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # loss에서 사용된 h와 h를 업데이트한 계산의 그래디언트 값을 더함
        dhraw = (1 - hs[t] * hs[t]) * dh # tanh 역전파
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam) # 그래디언트 발산 방지
            #-> 그래디언트 exploding(발산) 해결 방법 :
            #Simple heuristic solution: 발산할 때 마다 작은 수의 그래디언트 clip => 방지.
            #특정 임계 값에 도달 할 때마다 다음 알고리즘과 같이 작은 수로 다시 설정됩니다.
        
        return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    모델에서 지정된 글자 수(n) 만큼의 글자(숫자의 리스트)를 출력
    h 는 hidden state, seed_ix는 주어진 첫번째 글자
    """
    
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        # forward pass 수행
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))

        # 샘플링. 임의성을 부여하기 위해 argmax대신 array p에서 주어진 확률에 의해 하나의 문자를 선택
        ix = np.random.choice(range(vocab_size), p=p.ravel())

        #기존의 데이터에서 샘플링하기        
        # numpy.random.choice(a, size=None, replace=True, p=None)
        # a : 배열이면 원래의 데이터, 정수이면 range(a) 명령으로 데이터 생성
        # size : 정수. 샘플 숫자
        # replace : 불리언. True이면 한번 선택한 데이터를 다시 선택 가능
        # p : 배열. 각 데이터가 선택될 수 있는 확률


        # 다음 글자 추론을 위해 샘플링 된 글자를 다음 입력으로 사용 
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
    
        # 결과값 리스트에 추가
        ixes.append(ix)
    return ixes


n, p = 0, 0 #  반복 회수(n) 및 입력 데이터(p) 위치 초기화 

# Adagrad 알고리즘에 사용되는 메모리 변수 초기화
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) 
smooth_loss = -np.log(1.0/vocab_size)*seq_length # 학습이 이루어지기 전의 손실값

while True:
  # 입력데이터 준비, 텍스트의 맨 앞쪽부터 seq_length만큼씩 데이터를 준비
  # 데이터를 모두 사용하면 입력 데이터의 맨 처음으로 이동
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # RNN 메모리 초기화
        p = 0 # 입력 데이터의 맨 처음으로 이동
  
        # 입력(p~p+24번째 글자), 목표(p+1~p+25번째 글자) 데이터를 준비 
        inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
        targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # 학습을 100번 반복할 때마다 학습 결과를 출력
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200) #지금까지 학습한 RNN을 이용하여 숫자의 리스트를 출력
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

        
    # 손실함수에서 손실값과 그래디언트를 함께 계산
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # 반복횟수, 손실 출력
  


    # Adagrad 방식으로 파라미터 업데이트
    # Adagrad : 데이터-맞춤 학습속도 조정 방법 중 하나
    #
    # Assume the gradient dx and parameter vector x
    # cache += dx**2           
    # x += - learning_rate * dx / (np.sqrt(cache) + eps)
    
    # Adagrad(Adaptive Gradient)는 변수들을 update할 때 각각의 변수마다 step size를 다르게 설정해서 이동하는 방식이다.
    # cache는 그라디언트 벡터의 사이즈와 동일한 사이즈를 갖고 있다
    # 변수 eps는 분모가 너무 0에 가깝지 않도록 안정화 역할을 하고 주로 1e-4에서 1e-8의 값이 할당된다.
    # Adagrad의 단점이 있다면, 딥러닝의 경우에는, 학습 속도가 단조적이라 너무 한 방향으로 급진적(aggressive)으로 나가거나,
    # 혹은 학습을 너무 빨리 멈출 가능성도 있다.

    for param, dparam, mem in zip([Wxh,  Whh,  Why,  bh,  by],   # 가중치
                                [dWxh, dWhh, dWhy, dbh, dby],  # 그래디언트
                                [mWxh, mWhh, mWhy, mbh, mby]): # 메모리
        
        #zip 함수는 보통 List여러개로 slice 할때 사용을 합니다.
        #>>> zip([1,2,3], [4,5,6])  :   [(1, 4), (2, 5), (3, 6)]
        
        #zip("abc", "def")  :  [('a', 'd'), ('b', 'e'), ('c', 'f')]
        
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # 실제 파라메터 업데이트
        p += seq_length # 데이터 포인터를 seq_length만큼 우측으로 이동.
        n += 1 # 반복횟수 카운터

### LSTM으로 코드 변환 
lossFun 부분만 변경.=>vanishing gradient problem 해결

LSTM은 RNN의 hidden state에 cell-state를 추가한 구조입니다.

In [1]:
def lossFun(inputs, targets, hprev, cprev):
    xs, hs, cs, is_, fs, os, gs, ys, ps= {}, {}, {}, {}, {}, {}, {}, {}, {}
    hs[-1] = np.copy(hprev) # t=0일때 t-1 시점의 hidden state가 필요하므로
    cs[-1] = np.copy(cprev)
    loss = 0
    H = hidden_size
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        tmp = np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh  # hidden state. ###
        is_[t] = sigmoid(tmp[:H])  
        fs[t] = sigmoid(tmp[H:2 * H])  #Forget gate.
        #시그모이드 함수 : 값을 0과 1사이로 만들어 주기 위함. ###
        #그 값이 0이라면 이전 상태의 정보는 잊고 1이라면 이전 상태의 정보를 온전히 기억하게 됩니다
        os[t] = sigmoid(tmp[2 * H: 3 * H]) #Output gate.
        gs[t] = np.tanh(tmp[3 * H:])
        #gs[t]의 범위는 -1~1이기 때문에 각각 강도와 방향을 나타낸다고 이해함.
        #input gate = is_[t]⊙gs[t] 는 ‘현재 정보를 기억하기’ 위한 게이트. 
        cs[t] = fs[t] * cs[t-1] + is_[t] * gs[t] #Final memory cell
        hs[t] = os[t] * np.tanh(cs[t])  ## hidden state 업데이트 ##

        
    # compute loss
    for i in range(len(targets)):
        idx = len(inputs) - len(targets) + i
        ys[idx] = np.dot(Why, hs[idx]) + by  # unnormalized log probabilities for next chars
        ps[idx] = np.exp(ys[idx]) / np.sum(np.exp(ys[idx]))  # probabilities for next chars
        loss += -np.log(ps[idx][targets[i], 0])  # softmax (cross-entropy loss)

    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext, dcnext = np.zeros_like(hs[0]), np.zeros_like(cs[0])
    n = 1
    a = len(targets) - 1
    for t in reversed(range(len(inputs))):
        if n > len(targets):
            continue
        dy = np.copy(ps[t])
        dy[targets[a]] -= 1  # backprop into y 
        # 보통 tanh나 로지스틱 시그모이드를 사용.출력값의 범위를 제한해주면서 전 구간에서 미분 가능하기 때문에 backprop이 잘 적용됨.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext  # backprop into h
        
        ### rnn과 다른 부분
        dc = dcnext + (1 - np.tanh(cs[t]) * np.tanh(cs[t])) * dh * os[t]  # backprop through tanh nonlinearity
        dcnext = dc * fs[t]
        di = dc * gs[t]
        df = dc * cs[t-1]
        do = dh * np.tanh(cs[t])
        dg = dc * is_[t]
        
        #각각의 활성함수에 대한 로컬 그래디언트를 구해 흘러들어온 그래디언트를 곱해주면 됩니다.
        
        ddi = (1 - is_[t]) * is_[t] * di
        ddf = (1 - fs[t]) * fs[t] * df
        ddo = (1 - os[t]) * os[t] * do
        ddg = (1 - np.tanh(gs[t]) * np.tanh(gs[t])) * dg
        da = np.hstack((ddi.ravel(),ddf.ravel(),ddo.ravel(),ddg.ravel()))
        
        dWxh += np.dot(da[:,np.newaxis],xs[t].T)
        dWhh += np.dot(da[:,np.newaxis],hs[t-1].T)
        dbh += da[:, np.newaxis]
        dhnext = np.dot(Whh.T, da[:, np.newaxis])
        n += 1
        a -= 1
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)  # clip : 그래디언트 발산 방지
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1], cs[len(inputs) - 1]


#3개의 게이트(입력/출력/망각 게이트)는 각자 다른 가중치를 가지고 있으며
#그 값으로 각 게이트에 입력으로 들어오는 값을 조절합니다.

#forget gate(망각 게이트)의 bias를 1로 설정하는 팁을 활용하면 성능을 증가시킬 수도 있습니다. 
# (한편 Sutskever는 bias를 5로 설정하라고 권고한 바 있습니다.)



##생성(generative) 모델 : input문장과 유사한 새로운 문장을 생성.
#참고 : https://ratsgo.github.io/deep%20learning/2017/10/10/RNNsty/